In [1]:
import pandas as pd
# import pyspark.pandas as ps
import glob
import re
import string
import json
from tqdm import tqdm
from datetime import datetime
import os
import emoji

from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from spacy.lang.de.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [2]:
ins_raw_data = pd.read_csv("../../shared/Elections2024/Data/1_Processed_Raw/Instagram/insta_master.csv")
ins_raw_data.columns
# ins_data = ps.from_pandas(ins_data)

Index(['platformId', 'platform', 'date', 'updated', 'type', 'description',
       'postUrl', 'subscriberCount', 'score', 'media',
       'likeAndViewCountsDisabled', 'languageCode', 'legacyId', 'id',
       'statistics.actual.favoriteCount', 'statistics.actual.commentCount',
       'statistics.expected.favoriteCount', 'statistics.expected.commentCount',
       'account.id', 'account.name', 'account.handle', 'account.profileImage',
       'account.subscriberCount', 'account.url', 'account.platform',
       'account.platformId', 'account.verified', 'expandedLinks', 'imageText',
       'brandedContentSponsor.id', 'brandedContentSponsor.name',
       'brandedContentSponsor.handle', 'brandedContentSponsor.profileImage',
       'brandedContentSponsor.subscriberCount', 'brandedContentSponsor.url',
       'brandedContentSponsor.platform', 'brandedContentSponsor.platformId',
       'brandedContentSponsor.verified', 'account.pageDescription'],
      dtype='object')

In [3]:
ins_data = ins_raw_data[['platformId', 'id', 'date', 'updated', 'type', 'languageCode', 'description', 'postUrl']]
ins_data.to_csv('ins_data.csv', index=False)

In [4]:
ins_data.head(3)

,platformId,id,date,updated,type,languageCode,description,postUrl
0,3396384188624382912_4094415966,13907856|3396384188624382912,2024-06-22 23:56:08,2024-06-26 14:16:19,photo,es,#EN6 | Al menos 24 personas murieron el sábado...,https://www.instagram.com/p/C8iYWy7tavA/
1,3396383930473100898_1448725107,2475929|3396383930473100898,2024-06-22 23:55:37,2024-06-27 23:28:45,album,es,🇮🇱Decenas de miles de israelíes protestaron es...,https://www.instagram.com/p/C8iYTCgsbZi/
2,3396383103500594894_3074529058,11435815|3396383103500594894,2024-06-22 23:53:58,2024-06-25 18:04:22,photo,en,"Israeli strikes kill at least 42 in Gaza, encl...",https://www.instagram.com/p/C8iYHAVSq7O/


In [5]:
ins_raw_data.languageCode.value_counts()/len(ins_data)

en    0.553770
hi    0.079796
es    0.073119
ta    0.059394
ar    0.035316
        ...   
uk    0.000011
so    0.000011
qh    0.000011
sr    0.000011
tz    0.000011
Name: languageCode, Length: 87, dtype: float64

In [6]:
ins_data.description

0        #EN6 | Al menos 24 personas murieron el sábado...
1        🇮🇱Decenas de miles de israelíes protestaron es...
2        Israeli strikes kill at least 42 in Gaza, encl...
3        OT7 Quanny was spotted with Donald Trump in Ph...
4        Hoy Naama Levy cumple 20 años, pero por desgra...
                               ...                        
91654    I fear #netanyahou may cost #joebiden the elec...
91655    #biden #biden2024 #trump #trump2024 #illegal #...
91656    NEW TONIGHT👉 A federal judge refused Thursday ...
91657    Desde el 7 de octubre, Israel ha sido responsa...
91658    The American Conservative, a right-wing blog a...
Name: description, Length: 91659, dtype: object

# Eng Docs

In [7]:
ins_en = ins_data[ins_data.languageCode=='en']
ins_en.shape

(50758, 8)

In [8]:
test = ins_en.description[91656]
# test = 'A federal judge refused Thursday to throw out the classified documents prosecution of Donald Trump, turning aside defense arguments that a decades-old law permitted the former president to retain the sensitive records after he left office'
test

'NEW TONIGHT👉 A federal judge refused Thursday to throw out the classified documents prosecution of Donald Trump, turning aside defense arguments that a decades-old law permitted the former president to retain the sensitive records after he left office.\n\nlink in bio for more\n\n#Trump #classifieddocuments #maralago #florida #lawyers #2024election'

In [9]:
def find_hashtags(text):
    return re.findall(r"#[\w]+", text)

In [10]:
find_hashtags(test)

['#Trump',
 '#classifieddocuments',
 '#maralago',
 '#florida',
 '#lawyers',
 '#2024election']

In [11]:
def find_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.findall(text)

def emoji_to_text(emoji):
    return emoji.demojize(emoji)

def text_to_emoji(text):
    return emoji.emojize(text)

In [51]:
# help me: 轉換連續emoji
# ins_en['emoji_text'] = ins_en['description'].apply(lambda x: [emoji_to_text(e) for e in find_emojis(x)])

In [53]:
# help me: SettingWithCopyWarning
ins_en['hashtags'] = ins_en['description'].apply(lambda x: find_hashtags(x))

/tmp/ipykernel_249260/1694234090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ins_en['hashtags'] = ins_en['description'].apply(lambda x: find_hashtags(x))


<WordListCorpusReader in '.../corpora/stopwords' (not loaded yet)>


In [44]:
# for index, row in ins_en.iterrows():  # Using iterrows for safe iteration
#     hashtags = find_hashtags(row['description'])
#     emojis = [emoji_to_text(x) for x in find_emojis(row['description'])]
    
#     # Direct assignment to DataFrame using loc
#     ins_en.loc[index, 'hashtags'] = hashtags
#     ins_en.loc[index, 'emojis'] = emojis
# #     print(hashtags)
# #     print(emojis)

In [16]:
def pre_processing(content):
    # Processing the content using spaCy and replace punctuations with spaces
    punctuations = f"[{re.escape(string.punctuation + '’–—' + '“”')}]"
    content = re.sub(punctuations, " ", content)
    doc = nlp(content)

    tokens = [token.lemma_.lower() #lemmatize
              for token in doc
              if not token.is_stop #remove stopwords
              and not token.is_space #remove multiple spaces
        ]

    tokens = ' '.join(tokens).strip()

    return tokens

In [34]:
test_doc = [pre_processing(x) for x in ins_en['description'][:100]]

# TF-IDF

In [35]:
# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words=list(STOP_WORDS))

# Apply vectorizer to the documents
tfidf_matrix = vectorizer.fit_transform(test_doc)

In [36]:
# Print the feature names
print(vectorizer.get_feature_names_out())

# Print the TF-IDF matrix
print(tfidf_matrix.toarray())

['000' '06' '10' ... '𝗖𝗼𝘂𝗻𝘁𝗿𝗶𝗲𝘀' '𝗗𝗢𝗡𝗔𝗧𝗘' '𝟮𝟱']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [40]:
dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))


{'000': 4.005682604407159,
 '06': 4.921973336281314,
 '10': 4.921973336281314,
 '100': 4.921973336281314,
 '10mm': 4.921973336281314,
 '12': 4.921973336281314,
 '15': 4.51650822817315,
 '1500': 4.921973336281314,
 '17': 4.921973336281314,
 '19': 4.228826155721369,
 '20': 4.921973336281314,
 '2020': 4.921973336281314,
 '2022': 4.921973336281314,
 '2024': 2.976063187226001,
 '2024election': 4.921973336281314,
 '21': 4.921973336281314,
 '22': 4.921973336281314,
 '25': 4.921973336281314,
 '260th': 4.921973336281314,
 '27': 4.921973336281314,
 '2a': 4.228826155721369,
 '2ndamendment': 4.921973336281314,
 '30': 4.921973336281314,
 '37': 4.921973336281314,
 '39': 4.921973336281314,
 '42': 4.921973336281314,
 '45': 4.005682604407159,
 '47': 4.005682604407159,
 '4730560': 4.921973336281314,
 '4th': 4.921973336281314,
 '50': 4.921973336281314,
 '500': 4.51650822817315,
 '51': 4.921973336281314,
 '551': 4.921973336281314,
 '56': 4.921973336281314,
 '60': 4.921973336281314,
 '660': 4.9219733362813

In [29]:
len(vectorizer.get_feature_names_out())

1966

In [23]:
len(tfidf_matrix.toarray()[0])

1966

In [20]:
for x in vectorizer.get_feature_names_out():
    print(x)

000
06
10
100
10mm
12
15
1500
17
19
20
2020
2022
2024
2024election
21
22
25
260th
27
2a
2ndamendment
30
37
39
42
45
47
4730560
4th
50
500
51
551
56
60
660
69
76
7th
85
911
99
aa
able
abortionright
absolutely
abu
abut
accept
access
accord
account
accuse
achieve
achievement
acidart
acidhouse
acknowledgment
act
action
active
activism
activist
add
additionally
adopt
advertising
advocacy
afp
agentorange
aggrandizement
agree
ahhh
ahmedeldin
ai
aid
ain
air
aircraft
airforce
airman
ajlouni
al
alaqsa
alarming
alexei
alexeisayle
alexeisaylepod
alilivesmatter
aljazeeraenglish
allah
alleged
alleyesonrafah
alma
almigdadhassan0
alshaer
alshati
altuffah
alvin
alvinbragg
amanda
ambulance
amen
america
americafirst
american
americanfaithmedia
americanhistory
americanpolitics
americanpresident
americans
americawillbegreatagain
amid
amidst
amp
amplify
anchor
andonisia
andrade
andrewfeinstein
andrewtateinspiration
andrewtatemotivation
angry
announce
annual
anoint
anointed
anti
antiracism
antisemitism
antiw

In [24]:
def pre_processing(content):
    # Processing the content using spaCy and replace punctuations with spaces
    punctuations = f"[{re.escape(string.punctuation + '’–—' + '“”')}]"
    content = re.sub(punctuations, " ", content)
    doc = nlp(content)

    tokens = [token.lemma_.lower() #lemmatize
              for token in doc
              if not token.is_stop #remove stopwords
              and not token.is_space #remove multiple spaces
        ]

    tokens = ' '.join(tokens).strip()

    return tokens

100

In [34]:
ins_en['clean_description'] = ins_en['description'].apply(lambda x: pre_processing(x))

In [53]:
pre_processing(test)

'new tonight 👉 federal judge refuse thursday throw classify document prosecution donald trump turn aside defense argument decade old law permit president retain sensitive record leave office link bio trump classifieddocument maralago florida lawyer 2024election'